In [ ]:
# install packages
!pip install pandas
import pandas as pd
!pip install numpy
import numpy as np
!pip install holidays
import holidays
import os
from datetime import timedelta
#from google.colab import drive

# autogluon
!pip install autogluon
from autogluon.tabular import TabularPredictor

# visuals
!pip install sweetviz
import sweetviz as sv
!pip install seaborn
!pip install matplotlib
import seaborn as sns
import matplotlib.pyplot as plt

# evaluation metrics
# !pip install evaluate
# import evaluate

# display settings
pd.options.display.width = None
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 33.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [sweetviz]1/2 [sweetviz]

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


/usr/local/python/3.12.1/lib/python3.12/site-packages/sweetviz/graph.py:8: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


RuntimeError: operator torchvision::nms does not exist

# # **Import Data**
# https://www.transtats.bts.gov/OT_Delay/OT_DelayCause1.asp
# https://www.transtats.bts.gov/DL_SelectFields.aspx?gnoyr_VQ=FGJ&QO_fu146_anzr=b0-gvzr

In [7]:
# Import data
df = pd.read_parquet("/workspaces/build-with-ai-executing-and-evaluating-hugging-face-models-4077220/T_ONTIME_REPORTING1.parquet")
# df1 = pd.read_csv("c:\\Users\\18594\\Documents\\HuggingFace_LiL_Course\\T_ONTIME_REPORTING1.csv")
# df2 = pd.read_csv("c:\\Users\\18594\\Documents\\HuggingFace_LiL_Course\\T_ONTIME_REPORTING2.csv")

# # Combine dataframes
# df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10], ignore_index=True)

# Join to cancellation data. Note you are joining based on the ORIGIN airport!!!! NOTE: Could some feats come from destination???
#df = pd.merge(df,arp,how='left',left_on='ORIGIN',right_on='LocID')


# Sample the df for modeling
df = df.sample(n=10000, random_state=123)
df.columns = df.columns.str.lower() # make cols lower case
df.head()
df.head()

,year,quarter,month,day_of_month,day_of_week,fl_date,op_unique_carrier,op_carrier_airline_id,op_carrier,tail_num,...,div4_wheels_off,div4_tail_num,div5_airport,div5_airport_id,div5_airport_seq_id,div5_wheels_on,div5_total_gtime,div5_longest_gtime,div5_wheels_off,div5_tail_num
181188,2025,1,1,11,6,1/11/2025 12:00:00 AM,AA,19805,AA,N189UW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
344714,2025,1,1,20,1,1/20/2025 12:00:00 AM,UA,19977,UA,N24706,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
385780,2025,1,1,23,4,1/23/2025 12:00:00 AM,AA,19805,AA,N989NN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22986,2025,1,1,2,4,1/2/2025 12:00:00 AM,DL,19790,DL,N906AT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
387636,2025,1,1,23,4,1/23/2025 12:00:00 AM,DL,19790,DL,N329NB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Feature ideas
# seasonality - around holiday
# carrier
# origin/destination airport
# day of week
# take off time
# accuracy metrics by airline
# direction of travel - east to west, pick up on tailwinds?
# either departing or arrival airport is a hub # https://www.faa.gov/airports/planning_capacity/categories
# nulls!!!!!!!!
# budget v non budget airlines

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 539747 entries, 0 to 539746
Columns: 109 entries, YEAR to DIV5_TAIL_NUM
dtypes: float64(69), int64(21), object(19)
memory usage: 448.9+ MB


In [5]:
df.describe()

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,...,DIV4_WHEELS_OFF,DIV4_TAIL_NUM,DIV5_AIRPORT,DIV5_AIRPORT_ID,DIV5_AIRPORT_SEQ_ID,DIV5_WHEELS_ON,DIV5_TOTAL_GTIME,DIV5_LONGEST_GTIME,DIV5_WHEELS_OFF,DIV5_TAIL_NUM
count,539747.0,539747.0,539747.0,539747.000000,539747.000000,539747.000000,539747.000000,539747.000000,5.397470e+05,539747.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,2025.0,1.0,1.0,15.831995,4.019748,19953.522335,2492.102685,12666.054390,1.266609e+06,31744.371839,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,0.0,0.0,0.0,9.036782,1.925782,368.663980,1669.518460,1527.227187,1.527225e+05,1324.865984,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,2025.0,1.0,1.0,1.000000,1.000000,19393.000000,1.000000,10135.000000,1.013506e+06,30070.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,2025.0,1.0,1.0,8.000000,2.000000,19790.000000,1113.000000,11292.000000,1.129202e+06,30627.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,2025.0,1.0,1.0,16.000000,4.000000,19930.000000,2213.000000,12889.000000,1.288904e+06,31453.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,2025.0,1.0,1.0,24.000000,6.000000,20304.000000,3695.000000,14057.000000,1.405702e+06,32467.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,2025.0,1.0,1.0,31.000000,7.000000,20452.000000,8819.000000,16869.000000,1.686902e+06,35991.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Cancellation Rate
df['cancelled'].value_counts(normalize=True)

cancelled
0.0    0.9718
1.0    0.0282
Name: proportion, dtype: float64

In [9]:
# How many of each class
df['cancelled'].value_counts()

cancelled
0.0    9718
1.0     282
Name: count, dtype: int64

In [10]:
df.shape

(10000, 109)

In [11]:
# changes dtypes
df['cancelled'] = df['cancelled'].astype('category')
df['op_unique_carrier'] = df['op_unique_carrier'].astype('str')
df['op_carrier_airline_id'] = df['op_carrier_airline_id'].astype('str')
df['op_carrier'] = df['op_carrier'].astype('str')
df['op_carrier_fl_num'] = df['op_carrier_fl_num'].astype('str')

# if a col name starts with a given string, change dtype to string
for col in df.columns:
    if col.startswith('origin'):
        df[col] = df[col].astype('str')

for col in df.columns:
    if col.startswith('dest'):
        df[col] = df[col].astype('str')

df.dtypes

year                    int64
quarter                 int64
month                   int64
day_of_month            int64
day_of_week             int64
                       ...   
div5_wheels_on        float64
div5_total_gtime      float64
div5_longest_gtime    float64
div5_wheels_off       float64
div5_tail_num         float64
Length: 109, dtype: object

In [12]:
# holidays
# full list of supported holidays in over 50 countries -> https://pypi.org/project/holidays/
# NOTE: Any I want to manually add?????????????????????????????
US_holidays = holidays.US()

# preview list of reocgnized holidays for the years in our dataset
# you might want to append other relevant holidays that are not recognized in this list that may have an effect on sales
# such as Black Friday in the US or company/industry-specific holidays (ex: valentine's day may be more relevant for a jewler than a furniture store)
years = [2022,2023,2024,2025,2026]
for date in holidays.US(years=years).items():
  print(date)

# save the holidays as a separate string to compare against the df
US_Holidays = []
for date in holidays.US(years=years).items():
   US_Holidays.append(str(date[0]))

# create a separare date value type so we can compare to the holidays string list
df['fl_date_dt'] =  pd.to_datetime(df['fl_date'])
# Check if the dates in the 'date' column are holidays
df['ind_is_holiday'] = df['fl_date_dt'].apply(lambda x: US_holidays.get(x.date()) is not None) # marks w true false
# # mark date as 1 if it falls on a US holiday, zero otherwise
df['ind_is_holiday'] = np.where(df['ind_is_holiday']==True, 1.0, 0.0)

# Check if the dates in the 'date' column are within X days of a holiday
df['ind_is_near_holiday_down'] = df['fl_date_dt'].apply(lambda x: any((x.date() - timedelta(days=i)) in US_holidays for i in range(3)))
df['ind_is_near_holiday_up'] = df['fl_date_dt'].apply(lambda x: any((x.date() + timedelta(days=i)) in US_holidays for i in range(3)))

df['ind_is_near_holiday'] = np.where((df['ind_is_near_holiday_down']==True) | (df['ind_is_near_holiday_up']==True), 1.0, 0.0)
#df.head(50)

(datetime.date(2022, 1, 1), "New Year's Day")
(datetime.date(2022, 5, 30), 'Memorial Day')
(datetime.date(2022, 6, 19), 'Juneteenth National Independence Day')
(datetime.date(2022, 6, 20), 'Juneteenth National Independence Day (observed)')
(datetime.date(2022, 7, 4), 'Independence Day')
(datetime.date(2022, 9, 5), 'Labor Day')
(datetime.date(2022, 11, 11), 'Veterans Day')
(datetime.date(2022, 11, 24), 'Thanksgiving Day')
(datetime.date(2022, 12, 25), 'Christmas Day')
(datetime.date(2022, 12, 26), 'Christmas Day (observed)')
(datetime.date(2022, 1, 17), 'Martin Luther King Jr. Day')
(datetime.date(2022, 2, 21), "Washington's Birthday")
(datetime.date(2022, 10, 10), 'Columbus Day')
(datetime.date(2023, 1, 1), "New Year's Day")
(datetime.date(2023, 1, 2), "New Year's Day (observed)")
(datetime.date(2023, 5, 29), 'Memorial Day')
(datetime.date(2023, 6, 19), 'Juneteenth National Independence Day')
(datetime.date(2023, 7, 4), 'Independence Day')
(datetime.date(2023, 9, 4), 'Labor Day')
(date

/tmp/ipykernel_11774/3040624679.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['fl_date_dt'] =  pd.to_datetime(df['fl_date'])


In [13]:
# Calculate proximity to/from a holiday
def days_until_next_holiday(flight_date, holiday_calendar):

    # Sort holiday list so that the next holiday can be found
    future_holidays = sorted([d for d in holiday_calendar if d >= flight_date.date()])

    # If there is no next holiday, return None (this shouldn't happen since we included future years)
    if not future_holidays:
        return None

    # Find the next holiday date
    next_holiday = future_holidays[0]

    # Calculate days difference
    days_diff = (next_holiday - flight_date.date()).days
    return days_diff

df['days_until_holiday'] = df['fl_date_dt'].apply(lambda x: days_until_next_holiday(x, US_holidays))


def days_since_last_holiday(flight_date, holiday_calendar):

    # Sort holiday list so that the next holiday can be found
    past_holidays = sorted([d for d in holiday_calendar if d <= flight_date.date()])

    # If there is no next holiday, return None (this shouldn't happen since we included prior years)
    if not past_holidays:
        return None

    # Find the next holiday date
    last_holiday = past_holidays[0]

    # Calculate days difference
    days_diff = (flight_date.date() - last_holiday).days
    return days_diff

df['days_since_last_holiday'] = df['fl_date_dt'].apply(lambda x: days_since_last_holiday(x, US_holidays))
df.head()

,year,quarter,month,day_of_month,day_of_week,fl_date,op_unique_carrier,op_carrier_airline_id,op_carrier,tail_num,...,div5_longest_gtime,div5_wheels_off,div5_tail_num,fl_date_dt,ind_is_holiday,ind_is_near_holiday_down,ind_is_near_holiday_up,ind_is_near_holiday,days_until_holiday,days_since_last_holiday
181188,2025,1,1,11,6,1/11/2025 12:00:00 AM,AA,19805,AA,N189UW,...,NaN,NaN,NaN,2025-01-11,0.0,False,False,0.0,9,10
344714,2025,1,1,20,1,1/20/2025 12:00:00 AM,UA,19977,UA,N24706,...,NaN,NaN,NaN,2025-01-20,1.0,True,True,1.0,0,19
385780,2025,1,1,23,4,1/23/2025 12:00:00 AM,AA,19805,AA,N989NN,...,NaN,NaN,NaN,2025-01-23,0.0,False,False,0.0,25,22
22986,2025,1,1,2,4,1/2/2025 12:00:00 AM,DL,19790,DL,N906AT,...,NaN,NaN,NaN,2025-01-02,0.0,True,False,1.0,18,1
387636,2025,1,1,23,4,1/23/2025 12:00:00 AM,DL,19790,DL,N329NB,...,NaN,NaN,NaN,2025-01-23,0.0,False,False,0.0,25,22


# # **Feature Creations**

In [14]:
# Check for missing values, impute where necessary
df.isnull().sum()

year                        0
quarter                     0
month                       0
day_of_month                0
day_of_week                 0
                           ..
ind_is_near_holiday_down    0
ind_is_near_holiday_up      0
ind_is_near_holiday         0
days_until_holiday          0
days_since_last_holiday     0
Length: 116, dtype: int64

In [15]:
# get hour from departure and arrival times
df['scheduled_hour_of_departure'] = df['crs_dep_time'].astype(str).str.zfill(4).str.slice(0, 2).astype(int)
df['scheduled_hour_of_arrival'] = df['crs_arr_time'].astype(str).str.zfill(4).str.slice(0, 2).astype(int)

# indicator variables
df['ind_is_local'] = np.where(df['role\n(fy25)'] == 'Local', 1, 0) # maybe these airports dont have many resources to prevent cancellations if something goes wrong
df['ind_is_summer'] = np.where((df['month'] >= 6) & (df['month'] <= 8), 1, 0)
df['ind_late_scheduled_hour_of_departure'] = np.where((df['scheduled_hour_of_departure'] >= 20) & (df['scheduled_hour_of_departure'] <= 22), 1, 0)
df['ind_late_scheduled_hour_of_arrival'] = np.where((df['scheduled_hour_of_arrival'] >= 21) & (df['scheduled_hour_of_arrival'] <= 23), 1, 0)
df['ind_early_departure'] = np.where(df['dep_delay'] < 0, 1, 0)

df.head()

KeyError: 'role\n(fy25)'